<a href="https://colab.research.google.com/github/Masayoriosan/project/blob/main/project2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt



pwt10 = pd.read_stata('https://dataverse.nl/api/access/datafile/354098')

data = pwt10.loc[pwt10['country'].isin(["France","Germany","Canada","Italy","Japan","United Kingdom","United States"])][['year', 'countrycode', 'rgdpna', 'rkna', 'emp', 'labsh']]
data = data.loc[(data['year'] >= 1995) & (data['year'] <= 2019)].dropna()




data['a'] = 1 - data['labsh']  
data['KL'] = (data['rkna']**data['a']) * (data['emp']**(1 - data['a']))
data['A'] = data['rgdpna'] / ((data['rkna']**data['a']) * (data['emp']**(1 - data['a'])))  
data['Y'] = data['rgdpna'] 


data['logy'] = np.log(data['rgdpna'] / data['emp'])  
data['logk'] = np.log(data['KL'] / data['emp']) 
data['logA'] = np.log(data['A'])


data['dt'] = data['year'].diff()  

data['dlogy_dt'] = data['logy'].diff() / data['dt']
data['dlogk_dt'] = data['logk'].diff() / data['dt']
data['dlogA_dt'] = data['logA'].diff() / data['dt']


data['TFP_growth'] = data['dlogA_dt'] * 100  
data['capital_deepening'] = data['dlogk_dt'] * 100  
data['GDP_growth'] = data['dlogy_dt'] * 100  




data = data.dropna()


country_names = {
    'USA': 'United States',
    'JPN': 'Japan',
    'CAN': 'Canada',
    'DEU': 'Germany',
    'FRA': 'France',
    'GBR': 'United Kingdom',
    'ITA':'Italia',
}



summary = data.groupby('countrycode').agg({'GDP_growth': 'mean',
                                           'TFP_growth': 'mean',
                                           'capital_deepening': 'mean'})



summary['TFP寄与率'] = summary['TFP_growth'] / summary['GDP_growth'] 
summary['資本深化寄与率'] = summary['capital_deepening'] / summary['GDP_growth']

summary['countryname'] = summary.index.map(country_names)

summary = summary[['countryname', 'GDP_growth', 'TFP_growth', 'capital_deepening','TFP寄与率','資本深化寄与率']]



print(summary)


                countryname  GDP_growth  TFP_growth  capital_deepening  \
countrycode                                                              
CAN                  Canada    0.887700    0.648264           0.239437   
DEU                 Germany    0.662448    0.501324           0.161122   
FRA                  France    0.816542    0.212720           0.603822   
GBR          United Kingdom    1.036837   -0.274969           1.311810   
ITA                  Italia   -0.045964   -0.549236           0.503274   
JPN                   Japan    0.757615    0.721106           0.036508   
USA           United States    1.440635    0.835166           0.605465   

                TFP寄与率    資本深化寄与率  
countrycode                        
CAN           0.730274   0.269728  
DEU           0.756774   0.243222  
FRA           0.260513   0.739487  
GBR          -0.265200   1.265203  
ITA          11.949149 -10.949213  
JPN           0.951811   0.048188  
USA           0.579721   0.420277  
